In [1]:
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint, adfuller
import pandas as pd
import numpy as np
import itertools
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.pylab as pit
from matplotlib.pylab import rcParams
%matplotlib inline
from matplotlib import style
style.use('ggplot')
rcParams['figure.figsize'] = 20,10
import re

C:\Users\archana.parihar\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:

dfPairs = pd.DataFrame()
dfPairs = pd.read_csv('C:/Users/archana.parihar/Downloads/ResultsSnP/ListOfPairs_2_Medoid_Cluster_1.csv')
ListOfPairsToCheck = dfPairs['Pairs'].tolist()

In [3]:
df = pd.DataFrame()
df = pd.read_csv('C:/Users/archana.parihar/Downloads/allStocks16-17-18.csv')
ListOfSnPCompanies = df['Name'].tolist()
ListOfSnPCompanies = list(set(ListOfSnPCompanies))
print(ListOfSnPCompanies)

['KMI', 'WMB', 'EA', 'SYY', 'WRK', 'CHK', 'C', 'KR', 'IR', 'TSN', 'MON', 'CNP', 'CSCO', 'TSCO', 'WAT', 'NWS', 'NLSN', 'EIX', 'IPG', 'KSS', 'LUV', 'MKC', 'MSI', 'SPGI', 'ALGN', 'BLK', 'MHK', 'MMC', 'NFX', 'TRIP', 'PRGO', 'LEG', 'APH', 'HSY', 'USB', 'GM', 'VAR', 'DRI', 'AYI', 'BHGE', 'ULTA', 'SNPS', 'CL', 'BAX', 'JNJ', 'ANTM', 'ORCL', 'AON', 'GILD', 'PCG', 'EQIX', 'DISCA', 'IDXX', 'BMY', 'MOS', 'HBI', 'NEE', 'NVDA', 'MDT', 'GT', 'HIG', 'PHM', 'EXPD', 'LKQ', 'AMP', 'NUE', 'HLT', 'CAT', 'TXN', 'ADP', 'APTV', 'DG', 'FTI', 'AMGN', 'BEN', 'SHW', 'PSX', 'LEN', 'HRL', 'UPS', 'K', 'RL', 'SYF', 'FITB', 'WEC', 'DIS', 'CMA', 'ABBV', 'COTY', 'VRTX', 'ORLY', 'EMR', 'EW', 'AAL', 'GOOGL', 'AIV', 'UAA', 'RJF', 'ADM', 'DFS', 'IBM', 'CSX', 'CDNS', 'MTD', 'MET', 'BAC', 'FTV', 'IP', 'MS', 'AXP', 'COP', 'PAYX', 'V', 'CTSH', 'CF', 'WM', 'AMG', 'NBL', 'VNO', 'SPG', 'QCOM', 'PGR', 'MDLZ', 'XRX', 'WFC', 'NEM', 'EL', 'ETR', 'XEL', 'UA', 'ESS', 'AVB', 'BRKB', 'PM', 'AME', 'LOW', 'AOS', 'MRO', 'APC', 'TMK', 'ACN', 

In [4]:
for i in range(0,len(ListOfSnPCompanies)):
    if ListOfSnPCompanies[i]=='BRK.B':
        ListOfSnPCompanies[i] = 'BRKB'
    if ListOfSnPCompanies[i]=='BF.B':
        ListOfSnPCompanies[i] = 'BFB'
print(ListOfSnPCompanies)

['KMI', 'WMB', 'EA', 'SYY', 'WRK', 'CHK', 'C', 'KR', 'IR', 'TSN', 'MON', 'CNP', 'CSCO', 'TSCO', 'WAT', 'NWS', 'NLSN', 'EIX', 'IPG', 'KSS', 'LUV', 'MKC', 'MSI', 'SPGI', 'ALGN', 'BLK', 'MHK', 'MMC', 'NFX', 'TRIP', 'PRGO', 'LEG', 'APH', 'HSY', 'USB', 'GM', 'VAR', 'DRI', 'AYI', 'BHGE', 'ULTA', 'SNPS', 'CL', 'BAX', 'JNJ', 'ANTM', 'ORCL', 'AON', 'GILD', 'PCG', 'EQIX', 'DISCA', 'IDXX', 'BMY', 'MOS', 'HBI', 'NEE', 'NVDA', 'MDT', 'GT', 'HIG', 'PHM', 'EXPD', 'LKQ', 'AMP', 'NUE', 'HLT', 'CAT', 'TXN', 'ADP', 'APTV', 'DG', 'FTI', 'AMGN', 'BEN', 'SHW', 'PSX', 'LEN', 'HRL', 'UPS', 'K', 'RL', 'SYF', 'FITB', 'WEC', 'DIS', 'CMA', 'ABBV', 'COTY', 'VRTX', 'ORLY', 'EMR', 'EW', 'AAL', 'GOOGL', 'AIV', 'UAA', 'RJF', 'ADM', 'DFS', 'IBM', 'CSX', 'CDNS', 'MTD', 'MET', 'BAC', 'FTV', 'IP', 'MS', 'AXP', 'COP', 'PAYX', 'V', 'CTSH', 'CF', 'WM', 'AMG', 'NBL', 'VNO', 'SPG', 'QCOM', 'PGR', 'MDLZ', 'XRX', 'WFC', 'NEM', 'EL', 'ETR', 'XEL', 'UA', 'ESS', 'AVB', 'BRKB', 'PM', 'AME', 'LOW', 'AOS', 'MRO', 'APC', 'TMK', 'ACN', 

In [5]:
dat = {}
longEntryThr = -1
longCap = 0.5
shortEntryThr = 1
shortCap = -0.5
holdingPeriod = 5
done_pair = []
result = ""
pos_result = ""
neg_result = ""
mixed_result = ""
count = 0

#Making the dat dictionary 
for c in ListOfSnPCompanies:
    dfcomp = df.loc[df['Name'] == c]
    listComp = dfcomp['close'].values
    dat[c] = listComp

print(dat['BRKB'])
print(dat['BFB'])

[130.75 131.25 131.33 129.48 128.33 128.36 128.8  128.66 125.7  126.24
 127.44 126.56 126.78 127.72 126.58 125.09 128.07 136.94 136.73 136.84
 137.97 138.72 137.8  139.13 138.62 140.65 143.79 143.22 141.2  141.7
 140.81 141.06 140.99 142.4  146.89 144.83 144.24 143.88 144.62 142.91
 144.78 142.87 143.31 141.99 142.11 141.14 141.82 141.41 142.53 142.23
 141.76 143.96 142.23 142.51 141.91 143.65 143.93 144.67 143.93 143.31
 143.63 143.12 145.65 145.56 147.22 146.69 147.74 147.72 150.35 150.72
 150.43 149.58 149.78 146.62 148.97 143.76 143.17 144.15 143.87 144.52
 145.2  143.95 144.91 143.63 143.19 143.64 142.95 146.91 147.79 151.28
 156.06 156.92 160.12 159.39 160.21 161.34 164.69 164.11 164.86 166.12
 163.83 164.08 163.3  163.41 162.02 161.47 162.23 161.41 163.61 163.
 163.94 163.42 162.93 162.87 164.04 164.01 177.28 175.89 175.68 175.4
 175.3  174.97 175.37 174.98 166.87 166.86 165.83 166.47 166.17 166.35
 165.7  165.11 165.44 166.05 166.73 166.34 166.55 165.02 163.46 163.72
 163.27 16

In [6]:
listOfPairsFinal = []
for pair in ListOfPairsToCheck:
    pair = pair.replace('(',"")
    pair = pair.replace(')',"")
    pair = pair.replace(" ", "")
    pair = pair.split(',')
    pair[0] = re.sub('[^A-Za-z0-9]+', '', pair[0])    
    pair[1] = re.sub('[^A-Za-z0-9]+', '', pair[1])
    listOfPairsFinal.append(pair)

print(listOfPairsFinal[20])
print(listOfPairsFinal[20][1])
print(dat[listOfPairsFinal[20][1]][5])
#print(dat['RMD'])
print(listOfPairsFinal)
print(len(listOfPairsFinal))

['AAPL', 'RMD']
RMD
52.32
[['A', 'EW'], ['AAPL', 'ABBV'], ['AAPL', 'AET'], ['AAPL', 'ANTM'], ['AAPL', 'AVGO'], ['AAPL', 'BA'], ['AAPL', 'BAX'], ['AAPL', 'BIIB'], ['AAPL', 'CELG'], ['AAPL', 'CHTR'], ['AAPL', 'CME'], ['AAPL', 'COP'], ['AAPL', 'CSX'], ['AAPL', 'DIS'], ['AAPL', 'EOG'], ['AAPL', 'FDX'], ['AAPL', 'GILD'], ['AAPL', 'MDT'], ['AAPL', 'MMM'], ['AAPL', 'MS'], ['AAPL', 'RMD'], ['AAPL', 'SLB'], ['AAPL', 'SPGI'], ['AAPL', 'TWX'], ['AAPL', 'UNH'], ['AAPL', 'UNP'], ['AAPL', 'ZBH'], ['ABBV', 'ABT'], ['ABBV', 'ACN'], ['ABBV', 'AET'], ['ABBV', 'AMGN'], ['ABBV', 'AMT'], ['ABBV', 'AMZN'], ['ABBV', 'ANTM'], ['ABBV', 'ATVI'], ['ABBV', 'AVGO'], ['ABBV', 'AXP'], ['ABBV', 'BAC'], ['ABBV', 'BAX'], ['ABBV', 'BK'], ['ABBV', 'BLK'], ['ABBV', 'BRKB'], ['ABBV', 'C'], ['ABBV', 'CAT'], ['ABBV', 'CB'], ['ABBV', 'CELG'], ['ABBV', 'CL'], ['ABBV', 'CMCSA'], ['ABBV', 'CME'], ['ABBV', 'COP'], ['ABBV', 'COST'], ['ABBV', 'CSCO'], ['ABBV', 'CSX'], ['ABBV', 'CVS'], ['ABBV', 'CVX'], ['ABBV', 'DIS'], ['ABBV', 'EL'

In [7]:
import csv

count = 0
#s_and_p = 
#dat = {}
longEntryThr = -1
longCap = 0.5
shortEntryThr = 1
shortCap = -0.5
holdingPeriod = 5
done_pair = []
result = ""
pos_result = ""
neg_result = ""
mixed_result = ""

for pair in listOfPairsFinal:
    print(pair)
    in_market = 0
    short_pos = 0
    equity_curve = float(100)
    ret = []
    f_ret = []
    a_ret = []
    b_ret = []
    zdiff = []
    signal = [] #shod i long or short
    lCap5 = []
    sCap5 =[]
    age = [0]   #should be -1 indexed
    aged = []
    pos = [0]   #should be -1 indexed .. signal was +1 but long cap has been reached then pos = 0 .. im not in market
    for i in range (5,212):
        a_ret.append((dat[pair[0]][i]-dat[pair[0]][i-5])/dat[pair[0]][i-5])
        b_ret.append((dat[pair[1]][i]-dat[pair[1]][i-5])/dat[pair[1]][i-5])
    for j in range (59,206):
        zdiff.append(a_ret[j+1]/np.std(a_ret[0:j+1])-b_ret[j+1]/np.std(b_ret[0:j+1]))
    for k in range (65,211):
        f_ret.append((dat[pair[0]][k+1]-dat[pair[0]][k])/dat[pair[0]][k]-(dat[pair[1]][k+1]-dat[pair[1]][k])/dat[pair[1]][k])
    for l in range (0,len(zdiff)):
        if zdiff[l] < longEntryThr:
            signal.append(1)
        elif zdiff[l] > shortEntryThr:
            signal.append(-1)
        else:
            signal.append(0)

        if pos[l] == 1 and zdiff[l] > longCap:
            lCap5.append(1)
        else:
            lCap5.append(0)
        if pos[l] == -1 and zdiff[l] < shortCap:
            sCap5.append(1)
        else:
            sCap5.append(0)

        if signal[l]!=0:
            if pos[l] == 0:
                age.append(1)
            else:
                if pos[l]+signal[l] == 0:
                    age.append(1)
                else:
                    #check here if age should be age + 1
                    age.append(1)
        else:
            if age[l] == 5:
                age.append(0)
            else:
                if pos[l] == 1 and lCap5[l] != 0 or pos[l] == -1 and sCap5[l] != 0:
                    age.append(0)
                else:
                    if signal[l] + pos[l] != 0:
                        age.append(age[l] + 1)
                    else:
                        age.append(0)

        if age[l] == 5:
            if signal[l] == 0:
                aged.append(1)
            else:
                aged.append(0)
        else:
            aged.append(0)

        if signal[l] == 0:
            if aged[l] + sCap5[l] + lCap5[l] == 0:
                pos.append(pos[l])
            else:
                if aged[l] + sCap5[l] > 0 and pos[l] == -1:
                    pos.append(0)
                elif aged[l] + lCap5[l] > 0 and pos[l] == 1:
                    pos.append(0)
                else:
                    pos.append(pos[l])
        else:
            if signal[l]!=pos[l]:
                pos.append(signal[l])
            else:
                if aged[l] + sCap5[l] + lCap5[l] == 0:
                    pos.append(signal[l])
                else:
                    if aged[l] + sCap5[l] > 0 and pos[l] == -1  and signal[l] == -1:
                        pos.append(0)
                    elif aged[l] + lCap5[l] > 0 and pos[l] == 1 and signal[l] == 1:
                        pos.append(0)
                    else:
                        pos.append(signal[l])
    count = 0
        
    countOfPosResults = 0
    countOfNegResults = 0
    countOfTotalResults = 0
    
    for m in range(1,len(pos)-1):
        if(pos[m]*f_ret[m-1]>0):
            count = count + 1
        elif (pos[m]*f_ret[m-1]<0):
            count = count -1
        ret.append(pos[m]*f_ret[m-1])
        equity_curve = equity_curve*(1+pos[m]*f_ret[m-1])
        if pos[m]!=0:
            in_market = in_market + 1
        if pos[m]==-1:
            short_pos = short_pos + 1
    details = {}
    details["Sharpe"] = np.sqrt(260)*np.mean(ret)/np.std(ret)
    details["CumRet"] = (equity_curve-100)/100
    details["In_market"] = in_market/440
    details["Short"] = short_pos/440
    done_pair.append((pair[0], pair[1], details))

    if details["Sharpe"]>0 and details["CumRet"]>0:
        pos_result = pos_result  + str(done_pair[-1]) + "\n\n"
    elif details["Sharpe"]<0 and details["CumRet"]<0:
        neg_result = neg_result  + str(done_pair[-1]) + "\n\n"
    else:
        mixed_result = mixed_result  + str(done_pair[-1]) + "\n\n"
    result = result + str(done_pair[-1]) + "\n\n"

file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/analysis_2_Medoids_Cluster_1.txt', 'w')
file.write(result)
file.close()

file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/pos_analysis_2_Medoids_Cluster_1.txt', 'w')
file.write(pos_result)
file.close()

file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/neg_analysis_2_Medoids_Cluster_1.txt', 'w')
file.write(neg_result)
file.close()

file = open('C:/Users/archana.parihar/Downloads/ResultsSnP/mixed_analysis_2_Medoids_Cluster_1.txt', 'w')
file.write(mixed_result)
file.close()



['A', 'EW']
['AAPL', 'ABBV']
['AAPL', 'AET']
['AAPL', 'ANTM']
['AAPL', 'AVGO']
['AAPL', 'BA']
['AAPL', 'BAX']
['AAPL', 'BIIB']
['AAPL', 'CELG']
['AAPL', 'CHTR']
['AAPL', 'CME']
['AAPL', 'COP']
['AAPL', 'CSX']
['AAPL', 'DIS']
['AAPL', 'EOG']
['AAPL', 'FDX']
['AAPL', 'GILD']
['AAPL', 'MDT']
['AAPL', 'MMM']
['AAPL', 'MS']
['AAPL', 'RMD']
['AAPL', 'SLB']
['AAPL', 'SPGI']
['AAPL', 'TWX']
['AAPL', 'UNH']
['AAPL', 'UNP']
['AAPL', 'ZBH']
['ABBV', 'ABT']
['ABBV', 'ACN']
['ABBV', 'AET']
['ABBV', 'AMGN']
['ABBV', 'AMT']
['ABBV', 'AMZN']
['ABBV', 'ANTM']
['ABBV', 'ATVI']
['ABBV', 'AVGO']
['ABBV', 'AXP']
['ABBV', 'BAC']
['ABBV', 'BAX']
['ABBV', 'BK']
['ABBV', 'BLK']
['ABBV', 'BRKB']
['ABBV', 'C']
['ABBV', 'CAT']
['ABBV', 'CB']
['ABBV', 'CELG']
['ABBV', 'CL']
['ABBV', 'CMCSA']
['ABBV', 'CME']
['ABBV', 'COP']
['ABBV', 'COST']
['ABBV', 'CSCO']
['ABBV', 'CSX']
['ABBV', 'CVS']
['ABBV', 'CVX']
['ABBV', 'DIS']
['ABBV', 'EL']
['ABBV', 'EOG']
['ABBV', 'FDX']
['ABBV', 'GD']
['ABBV', 'GE']
['ABBV', 'GILD']
['

['AMGN', 'UTX']
['AMGN', 'VAR']
['AMGN', 'VZ']
['AMGN', 'WBA']
['AMGN', 'WFC']
['AMGN', 'WMT']
['AMGN', 'XOM']
['AMGN', 'ZBH']
['AMT', 'AMZN']
['AMT', 'ANTM']
['AMT', 'AVGO']
['AMT', 'AXP']
['AMT', 'BA']
['AMT', 'BAC']
['AMT', 'BAX']
['AMT', 'BDX']
['AMT', 'BIIB']
['AMT', 'BK']
['AMT', 'BLK']
['AMT', 'BRKB']
['AMT', 'BSX']
['AMT', 'C']
['AMT', 'CAT']
['AMT', 'CB']
['AMT', 'CHTR']
['AMT', 'CL']
['AMT', 'CMCSA']
['AMT', 'CME']
['AMT', 'COP']
['AMT', 'COST']
['AMT', 'CSCO']
['AMT', 'CSX']
['AMT', 'CVX']
['AMT', 'DIS']
['AMT', 'EL']
['AMT', 'EOG']
['AMT', 'EW']
['AMT', 'GD']
['AMT', 'GE']
['AMT', 'GM']
['AMT', 'GS']
['AMT', 'HD']
['AMT', 'HON']
['AMT', 'IBM']
['AMT', 'INTC']
['AMT', 'ISRG']
['AMT', 'JNJ']
['AMT', 'JPM']
['AMT', 'KO']
['AMT', 'LLY']
['AMT', 'LMT']
['AMT', 'LOW']
['AMT', 'MCD']
['AMT', 'MDT']
['AMT', 'MMM']
['AMT', 'MO']
['AMT', 'MON']
['AMT', 'MRK']
['AMT', 'MS']
['AMT', 'MSFT']
['AMT', 'NEE']
['AMT', 'NKE']
['AMT', 'NOC']
['AMT', 'OXY']
['AMT', 'PEP']
['AMT', 'PFE']
['AMT'

['BLK', 'GM']
['BLK', 'HD']
['BLK', 'HON']
['BLK', 'IBM']
['BLK', 'ISRG']
['BLK', 'JNJ']
['BLK', 'KO']
['BLK', 'LLY']
['BLK', 'LOW']
['BLK', 'MCD']
['BLK', 'MDT']
['BLK', 'MMM']
['BLK', 'MON']
['BLK', 'MRK']
['BLK', 'MSFT']
['BLK', 'NEE']
['BLK', 'NKE']
['BLK', 'NOC']
['BLK', 'OXY']
['BLK', 'PEP']
['BLK', 'PFE']
['BLK', 'PG']
['BLK', 'PM']
['BLK', 'QCOM']
['BLK', 'RMD']
['BLK', 'SCHW']
['BLK', 'SPGI']
['BLK', 'T']
['BLK', 'TMO']
['BLK', 'TWX']
['BLK', 'TXN']
['BLK', 'UNH']
['BLK', 'USB']
['BLK', 'UTX']
['BLK', 'VAR']
['BLK', 'VZ']
['BLK', 'WMT']
['BLK', 'XOM']
['BLK', 'ZBH']
['BRKB', 'CSCO']
['BRKB', 'CSX']
['BRKB', 'EW']
['BRKB', 'IDXX']
['BRKB', 'MDT']
['BRKB', 'MRK']
['BRKB', 'NEE']
['BRKB', 'PSX']
['BRKB', 'TWX']
['BRKB', 'UNH']
['BRKB', 'VAR']
['BRKB', 'WMT']
['BRKB', 'ZBH']
['BSX', 'ISRG']
['BSX', 'SPGI']
['BSX', 'UTX']
['C', 'CAT']
['C', 'CB']
['C', 'CELG']
['C', 'CHTR']
['C', 'CL']
['C', 'CMCSA']
['C', 'CME']
['C', 'COP']
['C', 'COST']
['C', 'CSCO']
['C', 'CSX']
['C', 'CVS']
['

['CSCO', 'ZBH']
['CSX', 'CVS']
['CSX', 'FDX']
['CSX', 'GILD']
['CSX', 'MCD']
['CSX', 'MDT']
['CSX', 'MMM']
['CSX', 'MON']
['CSX', 'MS']
['CSX', 'PG']
['CSX', 'QCOM']
['CSX', 'SPGI']
['CSX', 'TWX']
['CSX', 'UNH']
['CSX', 'UNP']
['CSX', 'WMT']
['CSX', 'ZBH']
['CVS', 'GD']
['CVS', 'LLY']
['CVS', 'OXY']
['CVS', 'RMD']
['CVS', 'TWX']
['CVS', 'UNP']
['CVS', 'WBA']
['CVS', 'WMT']
['CVX', 'EW']
['CVX', 'HD']
['CVX', 'HOLX']
['CVX', 'IDXX']
['CVX', 'INTC']
['CVX', 'ISRG']
['CVX', 'LLY']
['CVX', 'MON']
['CVX', 'NKE']
['CVX', 'PSX']
['CVX', 'SBUX']
['CVX', 'SYK']
['CVX', 'TJX']
['CVX', 'UNH']
['CVX', 'WBA']
['DIS', 'EL']
['DIS', 'FDX']
['DIS', 'GS']
['DIS', 'HOLX']
['DIS', 'HON']
['DIS', 'IBM']
['DIS', 'IDXX']
['DIS', 'JPM']
['DIS', 'KO']
['DIS', 'LLY']
['DIS', 'MCD']
['DIS', 'MDT']
['DIS', 'MON']
['DIS', 'MRK']
['DIS', 'MS']
['DIS', 'NEE']
['DIS', 'OXY']
['DIS', 'PEP']
['DIS', 'PFE']
['DIS', 'PG']
['DIS', 'PKI']
['DIS', 'PM']
['DIS', 'PNC']
['DIS', 'QCOM']
['DIS', 'SCHW']
['DIS', 'SLB']
['DIS', 

['LLY', 'TMO']
['LLY', 'TWX']
['LLY', 'TXN']
['LLY', 'UNH']
['LLY', 'UNP']
['LLY', 'UPS']
['LLY', 'USB']
['LLY', 'UTX']
['LLY', 'VAR']
['LLY', 'VZ']
['LLY', 'WBA']
['LLY', 'WFC']
['LLY', 'WMT']
['LLY', 'XOM']
['LLY', 'ZBH']
['LMT', 'NEE']
['LMT', 'OXY']
['LMT', 'PM']
['LMT', 'RTN']
['LMT', 'T']
['LMT', 'TJX']
['LMT', 'TXN']
['LMT', 'UNP']
['LMT', 'VAR']
['LMT', 'VZ']
['LMT', 'ZBH']
['LOW', 'OXY']
['LOW', 'RMD']
['LOW', 'UNH']
['MCD', 'MS']
['MCD', 'MSFT']
['MCD', 'NVDA']
['MCD', 'QCOM']
['MCD', 'TJX']
['MCD', 'TWX']
['MCD', 'UNP']
['MCD', 'WMT']
['MDLZ', 'MMM']
['MDLZ', 'MON']
['MDLZ', 'MRK']
['MDLZ', 'MS']
['MDLZ', 'MSFT']
['MDLZ', 'MU']
['MDLZ', 'NEE']
['MDLZ', 'NKE']
['MDLZ', 'OXY']
['MDLZ', 'PEP']
['MDLZ', 'PFE']
['MDLZ', 'PG']
['MDLZ', 'PM']
['MDLZ', 'PNC']
['MDLZ', 'PSX']
['MDLZ', 'QCOM']
['MDLZ', 'RMD']
['MDLZ', 'SBUX']
['MDLZ', 'SCHW']
['MDLZ', 'SLB']
['MDLZ', 'SYK']
['MDLZ', 'T']
['MDLZ', 'TJX']
['MDLZ', 'TWX']
['MDLZ', 'UNH']
['MDLZ', 'UNP']
['MDLZ', 'UPS']
['MDLZ', 'USB']
['

['VZ', 'WMT']
['VZ', 'XOM']
['VZ', 'ZBH']
['WBA', 'WFC']
['WBA', 'WMT']
['WBA', 'XOM']
['WBA', 'ZBH']
['WFC', 'WMT']
['WFC', 'XOM']
['WFC', 'ZBH']
['WMT', 'ZBH']
['XOM', 'ZBH']


In [8]:

listOfPosResult = pos_result.split('\n\n')
print(len(listOfPosResult))

listOfResult = result.split('\n\n')
print(len(listOfResult))

success = (len(listOfPosResult)/len(listOfResult))*100
print('Percentage of profiting pairs in Cluster: '+ str(success)+'%')

1361
2751
Percentage of profiting pairs in Cluster: 49.472918938567794%


In [9]:
list11 =[]
list22 = []
listRet = []
for i in range(0,len(listOfPosResult)-1): 
    list1 = listOfPosResult[i].split(',')
    list1[0] = list1[0].replace('(',"")
    list1[0] = re.sub('[^A-Za-z0-9]+', '', list1[0])    
    list1[1] = re.sub('[^A-Za-z0-9]+', '', list1[1])

    list2 = list1[3].split(':')
    cumR = list2[1].strip()
    
    list11.append(list1[0])
    list22.append(list1[1])
    listRet.append(float(cumR))
    
maxVal = max(listRet)

for i in range(0,len(listOfPosResult)-1): 
    list1 = listOfPosResult[i].split(',')
    list1[0] = list1[0].replace('(',"")
    list1[0] = re.sub('[^A-Za-z0-9]+', '', list1[0])    
    list1[1] = re.sub('[^A-Za-z0-9]+', '', list1[1])

    list2 = list1[3].split(':')
    cumR = list2[1].strip()
    cumR = float(cumR)
    
    if(cumR==maxVal):
        print(list1[0],list1[1],round(maxVal,2))

OXY QCOM 1.19


[0.1759639629364166, 0.08565275332838539, 0.2635370699059163, 0.40583018972149915, 0.08935056778715193, 0.1806152492298962, 0.04459658779850429, 0.42064956242201135, 0.009446671389486311, 0.0421071920799767, 0.3940682818870457, 0.4428703241152408, 0.4626343446323253, 0.07080154437660909, 0.29569782847056447, 0.20738104406916577, 0.2715351504786636, 0.5100923209204938, 0.018388802635257377, 0.07831855057980774, 0.21121578733470542, 0.5430300352809385, 0.828610643017287, 0.10721508346471338, 0.2072259899099535, 0.051408752124320785, 0.08280097535319612, 0.15059222433252514, 0.3494325284058493, 0.09008820569256698, 0.30245481961657505, 0.30677735341599344, 0.13199392154881537, 0.08761579495172356, 0.6137007585736174, 0.019604672480279105, 0.05954810240296169, 0.06297379044634013, 0.04649601435792249, 0.16291631760514746, 0.17549711950036057, 0.1410720809247576, 0.006828241145821607, 0.10002861781144133, 0.03672955769001547, 0.12572940091498183, 0.19989687647495302, 0.10799670871370864, 0.